In [57]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data = pd.read_pickle(r"../input/ford-data/data.pkl")

In [ ]:
data

In [ ]:
data['dim_0'] = data['dim_0'].apply(lambda x: x.reshape(500,1))

In [ ]:
X = data['dim_0']
y = data['labels']
y = y.astype(int)

In [ ]:
data.columns[0]

In [ ]:
for i in range(data.shape[0]):
    if True in np.isnan(data['dim_0'][i]).flatten():
        print(i)

In [ ]:
input_x = []
for array in X:
    input_x.append(array)

"""
X = pd.DataFrame(df.dim_0.tolist())
X = X.to_numpy()
"""

In [ ]:
input_x = np.array(input_x)

In [ ]:
input_x.shape

In [ ]:
test_time_series=input_x[0]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input_x,y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
n_classes = len(np.unique(y_train))

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
x_train.shape[1:]

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

In [ ]:
model.save("transformer_banana_muffin")

In [ ]:
scaler.fit(data['dim_0'])
data['dim_0'] = scaler.transform(data['dim_0'])

In [ ]:
!zip -r transformer.zip "/kaggle/working/transformer_banana_muffin"

# Normalize Data

In [ ]:
scaler.fit(data['dim_0'][0])

In [ ]:
data['dim_0'] = data['dim_0'].apply(lambda x: scaler.transform(x))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input_x,y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
n_classes = len(np.unique(y_train))

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

In [ ]:
model.save("transformer_banana_muffin_normalized")

In [56]:
!zip -r transformer_normalized.zip "/kaggle/working/transformer_banana_muffin_normalized"